# Imports

In [1]:
%reload_ext autoreload
%autoreload 2
import iris_utilities as iris

# Main

In [2]:
# Parse iris dataset and load data
eye_images = iris.parse_iris_dataset()

for eye_image in eye_images:
    # Process each eye image
    (image, image_id, label) = eye_image

    # Processing
    processed_image, radius, success = iris.processing(image, 60)

    # Remove reflection
    processed_image_no_reflection = iris.remove_reflection(processed_image)

    # Daugman
    sheet = iris.generate_rubber_sheet_model(processed_image)
    sheet_no_reflection = iris.generate_rubber_sheet_model(
        processed_image_no_reflection)

    # DWT features
    dwt_feature = iris.feature_extraction(sheet)
    dwt_feature_no_reflection = iris.feature_extraction(sheet_no_reflection)

    print("DWT Features:", dwt_feature)
    print("DWT Features (No Reflection):", dwt_feature_no_reflection)

    # Do further processing or save the results as needed

# Assuming you want to use DWT features for classification
features = [dwt_feature, dwt_feature_no_reflection]
labels = ['reflection', 'no_reflection']  # Example labels
X_train, X_test, y_train, y_test = iris.split_data(features, labels)

# Display the images
fig, axes = iris.plt.subplots(1, 2, figsize=(12, 5))
axes[0].imshow(processed_image_no_reflection)
axes[0].set_title('Iris Image (No Reflection)')
axes[0].axis('off')

axes[1].imshow(sheet_no_reflection)
axes[1].set_title('Daugman Rubber Sheet (No Reflection)')
axes[1].axis('off')

iris.plt.tight_layout()
iris.plt.show()

Label: S0001_F_30
Left eye count: 19
Right eye count: 45
Label: S0002_M_24
Left eye count: 14
Right eye count: 34
Label: S0003_M_32
Left eye count: 10
Right eye count: 20
Label: S0004_M_16
Left eye count: 25
Right eye count: 51
Label: S0005_M_11
Left eye count: 10
Right eye count: 30
Label: S0006_F_10
Left eye count: 12
Right eye count: 28
Label: S0007_F_23
Left eye count: 10
Right eye count: 31
Label: S0008_F_25
Dataset/VISA_Iris/VISA_Iris\S0008_F_25/L\1.bmp -> No circles (iris) found.
Error processing image: Dataset/VISA_Iris/VISA_Iris\S0008_F_25/L\1.bmp
Dataset/VISA_Iris/VISA_Iris\S0008_F_25/L\10.bmp -> No circles (iris) found.
Error processing image: Dataset/VISA_Iris/VISA_Iris\S0008_F_25/L\10.bmp
Dataset/VISA_Iris/VISA_Iris\S0008_F_25/L\2.bmp -> No circles (iris) found.
Error processing image: Dataset/VISA_Iris/VISA_Iris\S0008_F_25/L\2.bmp
Left eye count: 7
Right eye count: 32
Label: S0009_M_22
Left eye count: 28
Right eye count: 58
Label: S0010_M_24
Left eye count: 18
Right eye c

TypeError: Can't convert object to 'str' for 'filename'